# Batch Correlation Regularizer

In [1]:
import sys
sys.path.append('..')

In [2]:
from keras_bcr import BatchCorrRegulizer
import tensorflow as tf

class MyResNetBlock(tf.keras.layers.Layer):
    def __init__(self, n_units=128, bcr_rate=1e-6):
        super(MyResNetBlock, self).__init__()
        self.linear = tf.keras.layers.Dense(n_units)
        self.normal = tf.keras.layers.BatchNormalization()
        self.activ = tf.keras.layers.Activation('gelu')
        self.bcr = BatchCorrRegulizer(bcr_rate)
        self.add = tf.keras.layers.Add()
    def call(self, inputs):
        h = self.linear(inputs)
        h = self.normal(h)
        h = self.activ(h)
        h = self.bcr([h, inputs])  # before Add
        h = self.add([h, inputs])
        return h

In [3]:
n_features = 128

model = tf.keras.Sequential([
    MyResNetBlock(n_units=n_features, bcr_rate=1.),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.losses.MeanSquaredError()]
)

2022-05-09 21:24:32.591916: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
n_sample_size = 1000
X = tf.random.uniform((n_sample_size, n_features))  
y = tf.random.uniform((n_sample_size,))  

In [5]:
model.fit(X, y, epochs=10)

Epoch 1/10
32/32 [==============================] - 2s 3ms/step - loss: 1.1420 - mean_squared_error: 0.9846 - batch_corr_regulizer: 0.1574
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.5862 - mean_squared_error: 0.4243 - batch_corr_regulizer: 0.1595
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.4592 - mean_squared_error: 0.3010 - batch_corr_regulizer: 0.1592
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.3990 - mean_squared_error: 0.2498 - batch_corr_regulizer: 0.1558
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.3591 - mean_squared_error: 0.2057 - batch_corr_regulizer: 0.1554
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 0.3315 - mean_squared_error: 0.1782 - batch_corr_regulizer: 0.1568
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 0.3218 - mean_squared_error: 0.1694 - batch_corr_regulizer: 0.1568
Epoch 8/10
32/32 [=========